In [33]:
import os
from pathlib import Path
import ccxt
# import pandas as pd
import sys
import csv

market = 'future'
timeframe = '1d'




# -----------------------------------------------------------------------------

root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
sys.path.append(root + '/python')




# -----------------------------------------------------------------------------

# def fetchTickers(market) -> dict:
#         exchange = ccxt.binance({
#            'timeout': 10000,
#            'enableRateLimit': True,
#            'options': {
#                'defaultType': market,}
#        })
#         exchange.loadMarkets()

#         # Data container
#         ticker_list = []

#         #Fetch OHLCV data
#         ticker_list =  exchange.fetchTickers()

#         for key in list(filter(lambda x: '_' in x, ticker_list.keys())): del ticker_list[key]

#         # Return OHLCV data
#         return ticker_list


def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, data):
    p = Path("./data/" , market , '/', timeframe)
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)

# will need to increase limit if ever want to look at timeframe under 1h

def scrape_candles_to_csv(max_retries, timeframe, since, market, limit=100000):
    # instantiate the exchange by id
    exchange = ccxt.binance({
        'enableRateLimit': True,
        'options': {
        'defaultType': market,  }
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    ticker_list =  exchange.fetchTickers()
    # delete some funny tickers for future expiration
    for key in list(filter(lambda x: '_' in x, ticker_list.keys())): del ticker_list[key]
    # delete any key that doesn't have USDT
    for key in list(filter(lambda x: 'USDT' not in x, ticker_list.keys())): del ticker_list[key]
    # fetch all candles
    for key in ticker_list.keys():
        ohlcv = scrape_ohlcv(exchange, max_retries, key, timeframe, since, limit)
        # save them to csv file
        key = key.replace('/', '_')
        write_to_csv(key, ohlcv)
        print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', key+'.csv')

In [34]:
scrape_candles_to_csv(5, '1d', '2017-01-0100:00:00Z', 'future')

345 DGB/USDT candles in total from 2021-04-21T00:00:00.000Z to 2022-03-31T00:00:00.000Z
Saved 345 candles from 2021-04-21T00:00:00.000Z to 2022-03-31T00:00:00.000Z to DGB_USDT.csv
462 1INCH/USDT candles in total from 2020-12-25T00:00:00.000Z to 2022-03-31T00:00:00.000Z
Saved 462 candles from 2020-12-25T00:00:00.000Z to 2022-03-31T00:00:00.000Z to 1INCH_USDT.csv
581 DEFI/USDT candles in total from 2020-08-28T00:00:00.000Z to 2022-03-31T00:00:00.000Z
Saved 581 candles from 2020-08-28T00:00:00.000Z to 2022-03-31T00:00:00.000Z to DEFI_USDT.csv
430 ANKR/USDT candles in total from 2021-01-26T00:00:00.000Z to 2022-03-31T00:00:00.000Z
Saved 430 candles from 2021-01-26T00:00:00.000Z to 2022-03-31T00:00:00.000Z to ANKR_USDT.csv
65 1000BTTC/USDT candles in total from 2022-01-26T00:00:00.000Z to 2022-03-31T00:00:00.000Z
Saved 65 candles from 2022-01-26T00:00:00.000Z to 2022-03-31T00:00:00.000Z to 1000BTTC_USDT.csv
648 KNC/USDT candles in total from 2020-06-22T00:00:00.000Z to 2022-03-31T00:00:00.0